In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('../Data/raw/batting_stats_combined.csv')
pd.set_option('display.max_rows', None)

In [8]:
# Initial Inspection
print('Dataset shape: ', df.shape)
print('\nMissing Values:\n', df.isnull().sum())

Dataset shape:  (673, 321)

Missing Values:
 Unnamed: 0           0
IDfg                 0
Season               0
Name                 0
Team                 0
Age                  0
G                    0
AB                   0
PA                   0
H                    0
1B                   0
2B                   0
3B                   0
HR                   0
R                    0
RBI                  0
BB                   0
IBB                  0
SO                   0
HBP                  0
SF                   0
SH                   0
GDP                  0
SB                   0
CS                   0
AVG                  0
GB                   0
FB                   0
LD                   0
IFFB                 0
Pitches              0
Balls                0
Strikes              0
IFH                  0
BU                   0
BUH                  0
BB%                  0
K%                   0
BB/K                 0
OBP                  0
SLG                  0
OPS         

In [9]:
def clean_batting_stats(df):
    # Drop duplicates
    df = df.drop_duplicates()
    
    # Drop the unnamed index column if it exists
    if 'Unnamed: 0' in df.columns:
        df = df.drop('Unnamed: 0', axis=1)
    
    # Filter for minimum plate appearances (usually 100+ for meaningful stats)
    df = df[df['PA'] >= 100]
    
    # Create a unique player-season identifier
    df['player_season_id'] = df['Name'] + '_' + df['Season'].astype(str)
    
    # Handle missing values
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df[numeric_columns] = df[numeric_columns].fillna(0)
    
    
    return df

In [10]:
clean_df = clean_batting_stats(df)

clean_df.to_csv('../Data/processed/clean_batting_stats.csv', index=False)

In [19]:
from pybaseball import batting_stats
import pandas as pd

# Load our current batting stats
batting_df = pd.read_csv('../Data/processed/clean_batting_stats.csv')

# Get position data for each year
positions_list = []
for year in range(2019, 2024):
    print(f"Getting {year} position data...")
    year_data = batting_stats(year)
    print(year_data)
    # Keep only the columns we need for merging
    year_data = year_data[['Name', 'Team', 'Position', 'Season']]
    positions_list.append(year_data)

# Combine all years
positions_df = pd.concat(positions_list)

# Merge with our existing batting stats
merged_df = batting_df.merge(positions_df, 
                           on=['Name', 'Team', 'Season'],
                           how='left')

# Check results
print("\nPlayers without position data:")
print(merged_df[merged_df['Position'].isna()]['Name'].unique())

print("\nPosition Distribution:")
print(merged_df['Position'].value_counts())

# Save merged data
merged_df.to_csv('../Data/processed/batting_stats_with_positions.csv', index=False)

Getting 2019 position data...
      IDfg  Season                   Name   Team  Age    G   AB   PA    H  \
2    17678    2019           Alex Bregman    HOU   25  156  554  690  164   
4    15998    2019         Cody Bellinger    LAD   23  156  558  660  170   
1    10155    2019             Mike Trout    LAA   27  134  470  600  137   
5    12861    2019         Anthony Rendon    WSN   29  146  545  646  174   
0    11477    2019       Christian Yelich    MIL   27  130  489  580  161   
27   17350    2019          Rafael Devers    BOS   22  156  647  702  201   
31   12533    2019          Marcus Semien    OAK   28  162  657  747  187   
6    13613    2019            Ketel Marte    ARI   25  144  569  628  187   
7    12856    2019        George Springer    HOU   29  122  479  556  140   
9     9777    2019          Nolan Arenado    COL   28  155  588  662  185   
58   12916    2019       Francisco Lindor    CLE   25  143  598  654  170   
11   12161    2019        Xander Bogaerts    B